In [82]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


### 멜론 인기 테마 / 인기 장르 
- 메인 URL : https://www.melon.com/dj/tag/djtaghub_list.htm?tagSeq=테그ID
- PlayListURL : https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=리스트ID


##### 1. 인기테마/장르 별로 플레이리스트 ID가 저장된 파일을 불러온다.

In [83]:
df = pd.read_csv('data/멜론테그번호.CSV', encoding='euc-kr')
df.head()

,tag_name,id
0,기분전환,3
1,감성,73
2,힐링,5
3,드라이브,17
4,사랑,6


##### 2. 해당 ID별로 리스트 목록을 가지고 와서 각각 50개 내의 노래 리스트만 가지고온다.

In [84]:
plist_num = df.id.values
# 메인 URL
url = 'https://www.melon.com/dj/tag/djtaghub_list.htm?tagSeq='
# Sub URL
surl = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq='

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
driver = webdriver.Chrome('C:/Users/yonsai/Downloads/chromedriver_win32/chromedriver.exe')


In [85]:
plist1, plist2 = [], []

for i in plist_num:
    
    driver.get(url + str(i))
    time.sleep(1)


    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.service_list_play.d_djcol_list > ul > li > div > a')
    # print(len(lis))
    plist = [ li['href'].split(',')[-1][1:-3] for li in lis ]

    for pli in tqdm(plist):
        driver.get(surl+pli)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # plist1에 들어갈 요소들
        title = soup.select_one('.ellipsis.song_name').text.strip()
        tag = soup.select_one('.tag_list.type03').text.replace('#', '').replace('\n', ' ').strip()
        like = int(soup.select_one('.cnt').text.split()[-1].replace(',', '').strip())
        
        trs = soup.select('.service_list_song.d_song_list > table > tbody > tr' )
        
        song_ids = []
        # plist2에 들어갈 요소들
        for tr in trs:
            img = tr.select_one('img')['src']
            
            if tr.select_one('.bullet_icons.age_19') :
                continue
            tmp = tr.select_one('.rank01 > span > a')
            s_id = tmp['href'].split(',')[-1][:-2]
            song_ids.append(s_id)

            s_title = tmp.text.strip()
            s_artist = tr.select_one('.rank02 > span').text
            plist2.append({'plylstSeq': pli, 'songId': s_id, 'title': s_title, 'artist': s_artist, 'img': img})
        
        plist1.append({'plylstSeq': pli, 'title': title, 'tag': tag, 'like':like, 'songIds': song_ids})


df1 = pd.DataFrame(plist1)
df2 = pd.DataFrame(plist2)

print(df1.shape, df2.shape)


100%|██████████| 20/20 [00:44<00:00,  2.24s/it]

(760, 5) (33922, 5)


In [86]:
df1.tail(10)

,plylstSeq,title,tag,like,songIds
750,446335731,<클럽> 한번쯤은 들어봤을 클럽 노래 [2022.09.13],클럽 클럽노래 신나는 드라이브 기분전환 여행 추억의 EDM 힙합 그루브,6485,"[4298228, 4246992, 4092515, 4266494, 32928490,..."
751,438647664,"하체운동할때 듣는 폭발적인 트랩, 힙합 계속업뎃",운동 힙합 HipHop 헬스 트랩 파티 신나는 다이어트 기분전환 헬스장,2385,"[36388232, 36401062, 36420838, 36420524, 36286..."
752,489232898,반하기까지 단 6초! 감성 알앤비&힙합,힙합 세련된 알앤비 도입부 보컬 음색 감성 작업 트렌디 취향저격,688,"[32187697, 32159654, 32363484, 32860237, 32053..."
753,489524946,힙한 인스타감성 카페에서 흘러나오는 노래,힙한 카페 인스타 그루비 힙합,4338,"[36467631, 36447606, 36417475, 36381765, 36376..."
754,458150223,HIDDEN TRACKS 숨은 띵곡 '해외 랩/힙합' (매주 업데이트),힙합 히든 랩 HipHop 해외힙합 신나는 드라이브 띵곡 발굴 히든트랙,3357,"[36460740, 36453955, 36452636, 36452659, 36426..."
755,495471359,세상 힙한 국내 힙합 ᴘʟᴀʏʟɪsᴛ,국힙 힙합 국내힙합 그루비 HipHop groove,220,"[36469979, 36464011, 36124652, 32035984, 36376..."
756,503628248,내적 그루브 폭발 외힙 ᴘʟᴀʏʟɪsᴛ,힙합 그루브 국외 외힙 비트 Beat Trendy 트렌디 흥폭발 groove,302,"[36316152, 36452742, 32553874, 32883688, 35844..."
757,511765797,HIDDEN TRACKS 숨은 띵곡 '국내 랩/힙합' (매주 업데이트),힙합 히든 랩 HipHop 국내힙합 신나는 드라이브 띵곡 발굴 히든트랙,657,"[36450270, 36445519, 36462609, 36462610, 36465..."
758,481513250,시원함이 가득한 여름 재질 국내 힙합!,국내힙합 여름 힙합 휴가 여행 드라이브 기분전환 스트레스,129,"[32698151, 32734372, 35735549, 36417482, 31935..."
759,524871957,퇴사 마려울 때 듣는 국힙,회사 직장인 힘들때듣는노래 퇴사 스트레스 월요병 힘내요 짜증날때 힙합 노동요,24,"[34997085, 32556602, 33508136, 2612769, 342984..."


In [87]:
df2.tail(10)

,plylstSeq,songId,title,artist,img
33912,524871957,1731270,"덤벼라 세상아 (Feat. Joe Brown, 정현)",BILL STAX (빌스택스),https://cdnimg.melon.co.kr/cm/album/images/003...
33913,524871957,3847336,사나이,리듬파워,https://cdnimg.melon.co.kr/cm/album/images/021...
33914,524871957,3935149,Don't Hate Me,에픽하이 (EPIK HIGH),https://cdnimg.melon.co.kr/cm/album/images/021...
33915,524871957,30059882,"베짱이 (Feat. Car, the garden)",팔로알토 (Paloalto),https://cdnimg.melon.co.kr/cm/album/images/100...
33916,524871957,33388146,"Top Floor (Feat. MELOH, The Quiett)",TOIL,https://cdnimg.melon.co.kr/cm2/album/images/10...
33917,524871957,30725404,"퇴근 (Feat. 박재범, Crush)",스윙스,https://cdnimg.melon.co.kr/cm/album/images/101...
33918,524871957,7999021,또 하루 (Feat. 개코),개리,https://cdnimg.melon.co.kr/cm/album/images/026...
33919,524871957,3615521,넘버원,부가킹즈,https://cdnimg.melon.co.kr/cm/album/images/020...
33920,524871957,3565173,짠해 (Cheerz),사이먼 도미닉,https://cdnimg.melon.co.kr/cm/album/images/020...
33921,524871957,3624808,막잔하고 나갈게 (Lost One),다이나믹 듀오,https://cdnimg.melon.co.kr/cm/album/images/020...


In [88]:
df1.to_csv('data/playlist1.csv', index=False)
df2.to_csv('data/playlist2.csv', index=False)

In [90]:
df2.shape[0], df2.songId.nunique()

(33922, 13806)